In [ ]:
!pip install transformers

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving VOX_Videos_2023.csv to VOX_Videos_2023.csv


In [ ]:
import pandas as pd
from transformers import pipeline

# Load CSV file into a DataFrame with the correct delimiter
df = pd.read_csv('VOX_Videos_2023.csv', delimiter=';')

# Replace NaN values in 'videoDescription' with an empty string
df['videoDescription'] = df['videoDescription'].fillna('')

# Filter out rows where 'videoDescription' is empty
df = df[df['videoDescription'] != '']

# Create the sentiment classification pipeline
classifier = pipeline("text-classification", "clampert/multilingual-sentiment-covid19")

# Define a function to apply sentiment analysis and return the result
def analyze_sentiment(text):
    max_seq_length = 512
    truncated_text = text[:max_seq_length]

    result = classifier(truncated_text)
    label = result[0]['label']
    score = result[0]['score']
    probabilities = {label: score}

    # Extract probabilities for each label
    if 'scores' in result[0]:
        for res in result[0]['scores']:
            probabilities[res['label']] = res['score']

    return label, score, probabilities

# Apply the sentiment analysis to each text in the 'videoDescription' column
df['sentiment_output'], df['sentiment_score'], df['sentiment_probabilities'] = zip(*df['videoDescription'].map(analyze_sentiment))

# Display the DataFrame with the sentiment analysis results
display(df[['videoDescription', 'sentiment_output', 'sentiment_score', 'sentiment_probabilities']])

# Save the DataFrame to a CSV file
df[['videoDescription', 'sentiment_output', 'sentiment_score', 'sentiment_probabilities']].to_csv('VOX_2023_results.csv', index=False)

# Calculate overall probabilities
overall_probabilities = {'negative': 0, 'positive': 0}
total_rows = len(df)

for _, row in df.iterrows():
    probabilities = row['sentiment_probabilities']
    for label, probability in probabilities.items():
        overall_probabilities[label] += probability

# Normalize the probabilities to sum up to 100%
total_probability = sum(overall_probabilities.values())
for label in overall_probabilities:
    overall_probabilities[label] /= total_probability
    overall_probabilities[label] *= 100

# Display overall probabilities
print("Overall Probabilities:")
for label, probability in overall_probabilities.items():
    print(f"{label}: {probability:.2f}%")


,videoDescription,sentiment_output,sentiment_score,sentiment_probabilities
0,Desde VOX deseamos que todos los españoles enc...,positive,0.979686,{'positive': 0.979686439037323}
1,"El secretario general de VOX, Ignacio Garriga,...",positive,0.994244,{'positive': 0.9942435622215271}
2,"El presidente nacional de VOX, Santiago Abasca...",positive,0.939114,{'positive': 0.9391142725944519}
3,Discurso completo del presidente nacional de V...,positive,0.577394,{'positive': 0.577394425868988}
4,En Roma junto a nuestros amigos de Fratelli d'...,positive,0.998512,{'positive': 0.9985120892524719}
...,...,...,...,...
596,Entrevista al vicepresidente de la Junta de Ca...,positive,0.959770,{'positive': 0.9597699046134949}
597,https://www.voxespana.es/actualidad/vox-llevar...,negative,0.624226,{'negative': 0.6242256164550781}
598,Declaraciones de Santiago Abascal ante el Trib...,positive,0.950526,{'positive': 0.95052570104599}
599,"Santiago Abascal, presidente de VOX, y diputad...",positive,0.562319,{'positive': 0.5623193979263306}


Overall Probabilities:
negative: 12.80%
positive: 87.20%


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['sentiment_score'].plot(kind='hist', bins=20, title='sentiment_score')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('sentiment_output').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_7['sentiment_score'].plot(kind='line', figsize=(8, 4), title='sentiment_score')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_8['sentiment_output'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_8, x='index', y='sentiment_output', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_9['sentiment_output'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_9, x='sentiment_score', y='sentiment_output', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)